In [7]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
import pickle
import ts2vg
from diversity_measure import *

In [8]:
raw_data = loadarff("/home/lfrf/ic/ts_bakeoff_poc/data/LSST_TRAIN.arff")
df: pd.DataFrame = pd.DataFrame(raw_data[0])
df

,LSST_TRAIN,target
0,"[[17.597, 0.22974, -29.654, -59.21, -72.064, -...",b'6'
1,"[[2.4209, 1.6847, 0.50692, -0.26783, -0.02287,...",b'6'
2,"[[39.688, 163.99, 275.45, 300.36, 222.61, 98.1...",b'6'
3,"[[1.6554, 10.357, 17.141, 15.575, 8.0169, 4.06...",b'6'
4,"[[-15.139, -8.0405, -0.5869, 1.1502, -4.7361, ...",b'6'
...,...,...
2454,"[[16.989, -8.2046, -35.531, -43.974, -25.709, ...",b'95'
2455,"[[8.9566, 4.2787, 1.0259, 5.5224, 15.246, 20.9...",b'95'
2456,"[[503.12, 317.31, 242.74, 257.62, 115.73, 47.8...",b'95'
2457,"[[17.756, 26.275, 31.817, 32.078, 26.981, 18.9...",b'95'


In [9]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_basic_motions

def save_pickle(data: dict, path: str) -> None:
    # open a file, where you ant to store the data
    with open(path, 'wb') as file:
        pickle.dump(data, file)

In [10]:
# Creating graphs and calculating layer differences (only first 100 rows)
aux_df: pd.DataFrame = df.head(100)
for 


,LSST_TRAIN,target
0,"[[17.597, 0.22974, -29.654, -59.21, -72.064, -...",b'6'
1,"[[2.4209, 1.6847, 0.50692, -0.26783, -0.02287,...",b'6'
2,"[[39.688, 163.99, 275.45, 300.36, 222.61, 98.1...",b'6'
3,"[[1.6554, 10.357, 17.141, 15.575, 8.0169, 4.06...",b'6'
4,"[[-15.139, -8.0405, -0.5869, 1.1502, -4.7361, ...",b'6'
...,...,...
95,"[[13.423, 24.298, 32.751, 34.264, 27.905, 17.0...",b'15'
96,"[[14.102, 0.40264, -11.014, -11.232, -2.9754, ...",b'15'
97,"[[-3.5784, -9.0623, -12.483, -10.657, -3.9338,...",b'15'
98,"[[39.018, 24.73, 1.1615, -17.726, -22.59, -14....",b'15'
